In [131]:
import tensorflow as tf
import numpy as np
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.__version__

Num GPUs Available:  0


'2.3.1'

In [3]:
def read_data(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-4]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    data = np.array(data).transpose(0, 2, 1)
    d_shape = data.shape
    print(d_shape)
    return data.reshape(d_shape[0], 1, d_shape[1], d_shape[2])

def read_label(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-10]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    return np.array(data)

In [47]:
prefix_path = os.path.abspath(f'../../data/Dataset7')
train_data_path = os.path.abspath(f'{prefix_path}/train/train_data')
train_label_path = f'{prefix_path}/train/train_label'
test_data_path = f'{prefix_path}/test/test_data'
test_label_path = f'{prefix_path}/test/test_label'

train_data = read_data(train_data_path).transpose(0, 2, 3, 1) # 519
train_label = read_label(train_label_path)
test_data = read_data(test_data_path).transpose(0, 2, 3, 1) # 519
test_label = read_label(test_label_path)

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

# np.save("train_data.npy", train_data)
# np.save("train_label.npy", train_label)
# np.save("test_data.npy", test_data)
# np.save("test_label.npy", test_label)

(519, 6, 1024)
(58, 6, 1024)
(519, 6, 1024, 1)
(519, 1024)
(58, 6, 1024, 1)
(58, 1024)


In [133]:
# Training Parameters
num_steps = 150
batch_size = 32
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

def my_loss_fn(y_true, y_pred):
    return -tf.reduce_mean(input_tensor=y_true*tf.math.log(tf.clip_by_value(y_pred,1e-10,1.0))+(1-y_true)*tf.math.log(tf.clip_by_value(1-y_pred,1e-10,1.0)), name="cross_entropy")

In [120]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(64, [1, 16], activation='relu', input_shape=[6, 1024, 1], padding='same'),
    tf.keras.layers.Conv2D(64, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=[1, 2], strides=[1, 2], padding='same'),
    tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=[1, 2], strides=[1, 2], padding='same'),
    tf.keras.layers.Conv2D(256, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(128, [1, 2], strides=[1, 2], padding='same'),
    tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(64, [1, 2], strides=[1, 2], padding='same'),
    tf.keras.layers.Conv2D(64, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, [1, 16], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, [6, 1], activation='relu'),
    tf.keras.layers.Conv2D(1, [1, 1], activation='sigmoid'),
    tf.keras.layers.Reshape((-1,1024))
])

In [121]:
for layer in model.layers:
    print(layer.output_shape)

(None, 6, 1024, 64)
(None, 6, 1024, 64)
(None, 6, 512, 64)
(None, 6, 512, 128)
(None, 6, 512, 128)
(None, 6, 256, 128)
(None, 6, 256, 256)
(None, 6, 256, 256)
(None, 6, 256, 256)
(None, 6, 512, 128)
(None, 6, 512, 128)
(None, 6, 512, 128)
(None, 6, 1024, 64)
(None, 6, 1024, 64)
(None, 6, 1024, 64)
(None, 1, 1024, 256)
(None, 1, 1024, 1)
(None, 1, 1024)


In [128]:
model.compile(optimizer=optimizer,
              loss=my_loss_fn,
              metrics=['accuracy'])

In [129]:
model.fit(x=train_data, y=train_label, batch_size=batch_size, epochs=num_steps)

Epoch 1/150
14/17 [=======================>......] - ETA: 25s - loss: 0.6912 - accuracy: 0.0000e+00

KeyboardInterrupt: 